In [3]:
# This code prepares the Maven Coffee House data for analysis.

import pymysql
import pandas as pd

In [5]:
# The transactions, products, and stores tables are read from the original EXCEL file.

# The data is read using pandas and specifying their corresponding sheet names
transactions = pd.read_excel(
    "/Users/sylviaperez-montero/Desktop/DDEC_Bootcamp/project-2-eda-sql-master/data/maven_coffee_data.xlsx", 
    sheet_name="transactions"
)
stores = pd.read_excel(
    "/Users/sylviaperez-montero/Desktop/DDEC_Bootcamp/project-2-eda-sql-master/data/maven_coffee_data.xlsx", 
    sheet_name="stores"
)
products = pd.read_excel(
    "/Users/sylviaperez-montero/Desktop/DDEC_Bootcamp/project-2-eda-sql-master/data/maven_coffee_data.xlsx", 
    sheet_name="products"
)

In [6]:
# Does a preliminary check on each table's titles
print("Transactions columns:", transactions.columns)
print("Stores columns:", stores.columns)
print("Products columns:", products.columns)

Transactions columns: Index(['transaction_id', 'transaction_date', 'transaction_time',
       'transaction_qty', 'store_id', 'product_id', ' unit_price'],
      dtype='object')
Stores columns: Index(['store_id', 'location'], dtype='object')
Products columns: Index(['product_id', 'p_unit_price', 'product_category', 'product_type',
       'product_detail'],
      dtype='object')


In [7]:
# Drop duplicates in each table, to make sure there are no duplicate primary keys
transactions = transactions.drop_duplicates(subset='transaction_id')
stores = stores.rename(columns={'location': 'store_location'})
stores = stores.drop_duplicates(subset='store_id')
products = products.drop_duplicates(subset='product_id')

#Renames the prod_id variable in the database.
products = products.rename(columns={'product_id': 'prod_id'})

In [8]:
# Authentication
cnx = pymysql.connect(user='root', password='Bobby',
                      host='localhost', database='maven2')

In [13]:
# Checks whether the connection is open or closed so appropriate action can be taken.
if cnx.open:
    print("Connection open")
else:
    print("Connection is not successfully open")

# Creating cursor object to interact with database
cursor = cnx.cursor()

Connection open


In [15]:
# Creates the transactions table
query_transactions = """
CREATE TABLE IF NOT EXISTS maven2.transactions (
    transaction_id VARCHAR(20) PRIMARY KEY,
    transaction_date DATE NOT NULL,
    transaction_time VARCHAR(30) NOT NULL,
    transaction_qty VARCHAR(100) NOT NULL,
    store_id VARCHAR(10) NOT NULL,
    product_id VARCHAR(52) NOT NULL,
    unit_price DECIMAL(10, 2) NOT NULL
);
"""
cursor.execute(query_transactions)

0

In [17]:
# Creates the products table
query_products = """
CREATE TABLE IF NOT EXISTS maven2.products (
    prod_id VARCHAR(52) PRIMARY KEY,
    product_category VARCHAR(50) NOT NULL,
    product_type VARCHAR(50) NOT NULL,
    p_unit_price DECIMAL(10, 2) NOT NULL,
    product_detail VARCHAR(100) NOT NULL
);
"""
cursor.execute(query_products)

0

In [19]:
# Creates the stores table
query_stores = """
CREATE TABLE IF NOT EXISTS maven2.stores (
    store_id VARCHAR(10) PRIMARY KEY,
    store_location VARCHAR(50) NOT NULL
);
"""
cursor.execute(query_stores)


0

In [21]:
# Commits the changes for the table creation
cnx.commit()

In [23]:
# Adds the rows and entries to our tables
data = [transactions, stores, products]  # List of DataFrames
data_title = ['transactions', 'stores', 'products']  # Corresponding table names

# print(df.head())

In [25]:
#columns = transactions.columns

for df, table_name in zip(data, data_title):
    print(f"Columns in {table_name}: {df.columns.tolist()}")

# Adding the rows and entries to our tables
#for df, table_name in zip(data, data_title):
# Extract column names once for all rows
columns = ', '.join(df.columns)
placeholders = ', '.join(['%s'] * len(df.columns))

df = df.loc[:, ~df.columns.duplicated()]

# Construct the SQL query template
sql = f"INSERT IGNORE INTO maven2.{table_name} ({columns}) VALUES ({placeholders})"
    
for _, row in df.iterrows():
    # Convert row to a tuple to pass to the query
    cursor.execute(sql, tuple(row))

# Commit changes
cnx.commit()

# Close the connection
#cursor.close()
#cnx.close()

print("Database operations completed successfully.")

Columns in transactions: ['transaction_id', 'transaction_date', 'transaction_time', 'transaction_qty', 'store_id', 'product_id', ' unit_price']
Columns in stores: ['store_id', 'store_location']
Columns in products: ['prod_id', 'p_unit_price', 'product_category', 'product_type', 'product_detail']
Database operations completed successfully.


In [27]:
# Finally, add the rows and entries to our tables
data = [products, transactions, stores]
data_title = ['products', 'transactions', 'stores']

table = table.loc[:, ~table.columns.duplicated()]

# Loop over tables and data
for j in range(len(data)):    
    table = data[j]  # Get the table
    columns = table.columns  # Get column names
    parameter = ", ".join(list(columns))  # Create a string for the list of columns
    param_2 = ("%s, " * len(columns))[:-2]  # Create the placeholder for the values
    
    # Create the query for the table
    query = f"INSERT IGNORE INTO maven2.{data_title[j]} ({parameter}) VALUES ({param_2})"
    
    for i in range(len(data[j])):
        row = tuple(table.iloc[i])  # Convert row to a tuple
        try:
            # Execute the query with row data
            cursor.execute(query, row)
        except Exception as e:
            print(f"Error inserting row {row} into {data_title[j]}: {e}")

# Commit changes
cnx.commit()

# Close the connection
cursor.close()
cnx.close()

print("Database operations completed successfully.")

NameError: name 'table' is not defined

In [29]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_id    149116 non-null  int64         
 1   transaction_date  149116 non-null  datetime64[ns]
 2   transaction_time  149116 non-null  object        
 3   transaction_qty   149116 non-null  int64         
 4   store_id          149116 non-null  int64         
 5   product_id        149116 non-null  int64         
 6    unit_price       149116 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 8.0+ MB


In [32]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   prod_id           80 non-null     int64  
 1   p_unit_price      80 non-null     float64
 2   product_category  80 non-null     object 
 3   product_type      80 non-null     object 
 4   product_detail    80 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 3.3+ KB


In [34]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   store_id        3 non-null      int64 
 1   store_location  3 non-null      object
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


In [38]:
transactions.isnull().sum()

transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
product_id          0
 unit_price         0
dtype: int64

In [40]:
products.isnull().sum()

prod_id             0
p_unit_price        0
product_category    0
product_type        0
product_detail      0
dtype: int64

In [42]:
stores.isnull().sum()

store_id          0
store_location    0
dtype: int64